## Vanilla GAN on MNIST Dataset

In [12]:
import tensorflow as tf
import keras
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers
from keras.layers.convolutional import UpSampling2D, Conv2D
import numpy as np
import matplotlib.pyplot as plt

In [13]:
class CGAN():
    def __init__(self):
        # Input Shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10   # we have 10 classes in our dataset, digits from 0-9
        self.latent_dim = 100
        optimizer = Adam(0.0002, 0.5)   
        # Learning Rate = 0.0002
        # Momentum Parameter = 0.5

        # Compiling the Discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])
        self.discriminator.trainable = False

        # Generator operations
        self.generator = self.build_generator()
        # self.generator.compile(loss=['binary_crossentropy'],
        #     optimizer=optimizer,
        #     metrics=['accuracy'])
        # we have Adam optimizer and we use binary_cross_entropy cause an image can either be real or fake

        noise = Input(shape=(self.latent_dim,))  # generating some noise as we want our generator to generate some images
        img = self.generator(noise)
        valid = self.discriminator(img)  # feeding the discriminator with images generated by generator
        self.combined = Model(noise,valid)
        # This line creates a Keras model object that takes the generator input noise as input and outputs the discriminator's classification of the generated images.
        # This is the combined model of the vanilla GAN, where the generator and discriminator are trained together.
        
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim, kernel_initializer = 'uniform', bias_initializer = 'zeros'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(720))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        # Batch Normalization will help us to find the minimum of loss function more easily, batch normlizartion main batches ke according,
        # values ko normalize kardiya jaata hai, kuch gradient descent wala scene. Learning rate vaghera vaghera larger use honge to get the optimum faster.
        # 1) Speeds up training
        # 2)Activations mein - Allows sub-optimal starts. -1 aur 1 ke beech se start krenge toh jaldi hee minimum pe pahuch jayenge.
        # 3) Dropout can be evacuated for Regularisation

        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))
        # our final shape should be (28,28,1) 
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        # feeding the generator with some noise
        model_input = noise
        img = model(model_input)
        
        # Overall, by defining the generator network as a Keras model that takes a noise vector as input and generates an image as output, 
        # we can train the generator to generate images that are indistinguishable from real images.

        return Model(noise, img)
        # Here input is noise and output is image 
    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(1024, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        # We use Leaky Relu instead of Relu because of dying Relu activations
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        flat_img = Flatten()(img)
        validity = model(flat_img)
        # By feeding the flattened image tensor to the discriminator network, we are effectively treating each pixel in the image as a separate feature
        #  and passing it through the discriminator network. The discriminator then uses this information to classify the image as real or fake. 
        #  This is a common approach in GANs, where the input to the discriminator network is often a flattened image tensor or a feature vector extracted from the image.

        return Model(img, validity)
        # here Input=img and Output=validity (flattened image)

    def train(self, epochs, batch_size=128, sample_interval=50):
        (X_train, y_train), (_, _) = mnist.load_data()
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs= X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, 100))
            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            g_loss = self.combined.train_on_batch(noise, valid)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].set_title("Image")
                axs[i,j].axis('off')
                cnt += 1
        plt.savefig("images%d.png" % epoch)
        plt.close()

In [14]:
if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=10000, batch_size=128, sample_interval=200)

Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 11ms/step
7506 [D loss: 0.667239, acc.: 59.38%] [G loss: 0.894096]
4/4 [==============================] - 0s 12ms/step
7507 [D loss: 0.671871, acc.: 57.81%] [G loss: 0.844059]
4/4 [==============================] - 0s 11ms/step
7508 [D loss: 0.641118, acc.: 62.50%] [G loss: 0.896612]
4/4 [==============================] - 0s 13ms/step
7509 [D loss: 0.667085, acc.: 57.42%] [G loss: 0.894285]
4/4 [==============================] - 0s 11ms/step
7510 [D loss: 0.673944, acc.: 56.64%] [G loss: 0.923603]
4/4 [==============================] - 0s 12ms/step
7511 [D loss: 0.659573, acc.: 62.11%] [G loss: 0.936118]
4/4 [==============================] - 0s 11ms/step
7512 [D loss: 0.676966, acc.: 58.20%] [G loss: 0.861120]
4/4 [==============================] - 0s 16ms/step
7513 [D loss: 0.658909, acc.: 58.59%] [G loss: 0.872514]
4/4 [==============================] - 0s 12ms/step
7514 [D loss: 0.666236, a